In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('경상대_날씨\경상대_일자별관측.csv')

# 전처리 column
# WD, WS, PA, PS, TA, TD, HM, PV, RN, RN.1, RN.2, CA, CA.1, CH, VS, SS, SI, TS

data.shape

(26255, 46)

In [4]:
# 전처리할 칼럼 리스트
columns_to_process = ['WD', 'WS', 'PA', 'PS', 'TA', 'TD', 'HM', 
                     'PV', 'RN', 'RN.1', 'RN.2', 'CA', 'CA.1', 
                     'CH', 'VS', 'SS', 'SI', 'TS']

In [5]:
def preprocess_weather_data(df, columns_to_process):
    """
    기상 데이터의 결측값을 처리하고 스플라인 보간을 수행하는 함수
    """
    df_processed = df.copy()
    
    print("\nProcessing columns:")
    for column in columns_to_process:
        if column in df_processed.columns:
            print(f"\nProcessing {column}:")
            print(f"Missing values before: {df_processed[column].isin([-9, -99]).sum()}")
            
            # -9와 -99를 NaN으로 변환
            df_processed[column] = df_processed[column].replace([-9, -99], np.nan)
            
            # 결측값이 있는 경우에만 보간 수행
            if df_processed[column].isna().any():
                # 인덱스 배열 생성
                x = np.arange(len(df_processed))
                y = df_processed[column].values
                
                # NaN이 아닌 값들의 인덱스와 값 추출
                mask = ~np.isnan(y)
                x_known = x[mask]
                y_known = y[mask]
                
                # 최소 4개의 데이터 포인트가 필요
                if len(x_known) > 3:
                    # 스플라인 보간 수행
                    cs = CubicSpline(x_known, y_known)
                    y_interpolated = cs(x)
                    df_processed[column] = y_interpolated
                else:
                    # 데이터가 부족한 경우 선형 보간으로 대체
                    df_processed[column] = df_processed[column].interpolate(method='linear')
            
            print(f"Missing values after: {df_processed[column].isna().sum()}")
    
    return df_processed


In [6]:
# 전처리 수행
processed_data = preprocess_weather_data(data, columns_to_process)


Processing columns:

Processing WD:
Missing values before: 5
Missing values after: 0

Processing WS:
Missing values before: 5
Missing values after: 0

Processing PA:
Missing values before: 4
Missing values after: 0

Processing PS:
Missing values before: 4
Missing values after: 0

Processing TA:
Missing values before: 7
Missing values after: 0

Processing TD:
Missing values before: 82
Missing values after: 0

Processing HM:
Missing values before: 26
Missing values after: 0

Processing PV:
Missing values before: 26
Missing values after: 0

Processing RN:
Missing values before: 24105
Missing values after: 0

Processing RN.1:
Missing values before: 20231
Missing values after: 0

Processing RN.2:
Missing values before: 20231
Missing values after: 0

Processing CA:
Missing values before: 46
Missing values after: 0

Processing CA.1:
Missing values before: 260
Missing values after: 0

Processing CH:
Missing values before: 10936
Missing values after: 0

Processing VS:
Missing values before: 35

In [7]:
# 결과 확인
print("\nPreprocessing complete!")
print(f"Final data shape: {processed_data.shape}")


Preprocessing complete!
Final data shape: (26255, 46)


In [8]:
# 전처리된 데이터 저장
processed_data.to_csv('경상대_날씨/경상대_일자별관측_processed.csv', index=False)
print("\nProcessed data saved to: 경상대_날씨/경상대_일자별관측_processed.csv")


Processed data saved to: 경상대_날씨/경상대_일자별관측_processed.csv
